This notebook will serve as the master notebook which the TAs can run from start to finish

In [2]:
import pandas as pd
import numpy as np
from src.utils import get_batches, shuffle, train_val_split


%load_ext autoreload
%autoreload 2

In [3]:
# Define the path to the data
data_path = './data/essay_df.pkl'
df = pd.read_pickle(data_path)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8764 entries, 0 to 10683
Data columns (total 8 columns):
essay_id         8764 non-null int64
essay_set        8764 non-null int64
essay            8764 non-null object
domain1_score    8764 non-null int64
essays_embed     8764 non-null object
word_count       8764 non-null int64
max_score        8764 non-null float64
norm_score       8764 non-null float64
dtypes: float64(2), int64(4), object(2)
memory usage: 616.2+ KB


In [5]:
X = np.array(df['essays_embed'])
y = np.array(df['norm_score'])
sets = np.array(df['essay_set'])

In [6]:
X_stacked = np.stack(X, axis=0)
print(X_stacked.shape)

(8764, 600, 100)


In [7]:
X_flat = np.reshape(X_stacked, [X_stacked.shape[0], -1])
print(X_flat.shape)

(8764, 60000)


In [128]:
# instead of flattening, take the average value of each part of vectorized word in essay
#X_flat = np.mean(X_stacked, axis = 1)

In [8]:
X_shuffled, y_shuffled, sets_shuffled = shuffle(X_flat, y, sets)

In [9]:
print(X_shuffled.shape)

(8764, 60000)


In [10]:
X_train, y_train, s_train, X_val, y_val, s_val = train_val_split(X_shuffled, y_shuffled, sets_shuffled, train_prop=0.8)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape )

[5 6 5 ... 4 1 5]
(7012, 60000) (1752, 60000) (7012,) (1752,)


In [11]:
from src.mlp import MLP
input_dim = X_train.shape[1]
batch_size = 10

batch_gen = get_batches(X_train, y_train, s_train, batch_size, net_type='mlp')

my_net = MLP(input_dim=input_dim, hidden_dims=[100, 100], num_classes=12, reg=False, l2_reg=1e-4)

In [12]:
my_net.train(gen=batch_gen, X_val=X_val, y_val=y_val, s_val=s_val, n_epochs=20, lr=1e-4)

loss for counter 10 is 2.4922592639923096
counter 10: valid acc = 0.2083333283662796
[3 3 3 8 8 8 2 3 3 3 3 2 3 3 3 2 3 2 8 3]
[4 1 1 8 8 5 1 2 3 3 1 2 2 3 1 1 0 1 8 3]


loss for counter 20 is 2.4893555641174316
counter 20: valid acc = 0.21289955079555511
[3 3 3 8 8 8 2 3 3 3 3 2 3 3 1 2 3 1 8 3]
[4 1 1 8 8 5 1 2 3 3 1 2 2 3 1 1 0 1 8 3]


loss for counter 30 is 1.9858654737472534
counter 30: valid acc = 0.2083333283662796
[3 3 3 8 8 8 2 3 3 3 3 2 3 3 3 2 3 2 8 3]
[4 1 1 8 8 5 1 2 3 3 1 2 2 3 1 1 0 1 8 3]


loss for counter 40 is 1.711927056312561
counter 40: valid acc = 0.1997716873884201
[1 1 1 8 8 4 1 1 1 1 1 1 1 1 1 1 1 1 8 1]
[4 1 1 8 8 5 1 2 3 3 1 2 2 3 1 1 0 1 8 3]


loss for counter 50 is 1.880250096321106
counter 50: valid acc = 0.17922374606132507
[1 1 1 8 8 4 1 1 1 1 1 1 1 1 1 1 1 1 8 1]
[4 1 1 8 8 5 1 2 3 3 1 2 2 3 1 1 0 1 8 3]


...

loss for counter 540 is 1.3759324550628662
counter 540: valid acc = 0.3133561611175537
[3 1 1 8 9 8 2 3 3 3 1 1 3 1 1 1 1 1 8 3]
[4 1 1 8 8 5 1 2 3 3 1 2 2 3 1 1 0 1 8 3]


loss for counter 550 is 1.4562139511108398
counter 550: valid acc = 0.3430365324020386
[3 1 1 9 9 8 2 3 3 3 1 2 2 2 1 1 1 1 9 3]
[4 1 1 8 8 5 1 2 3 3 1 2 2 3 1 1 0 1 8 3]


loss for counter 560 is 1.3627638816833496
counter 560: valid acc = 0.2871004641056061
[3 1 1 8 9 8 2 3 3 3 1 1 1 1 1 1 1 1 8 3]
[4 1 1 8 8 5 1 2 3 3 1 2 2 3 1 1 0 1 8 3]


loss for counter 570 is 1.1372416019439697
counter 570: valid acc = 0.31107306480407715
[3 1 1 9 9 8 2 3 3 3 1 1 1 1 1 1 1 1 9 3]
[4 1 1 8 8 5 1 2 3 3 1 2 2 3 1 1 0 1 8 3]


KeyboardInterrupt: 